<a href="https://colab.research.google.com/github/Sagargupta16/LeetCode_Rating_Predictor/blob/main/LC_Contest_Rating_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import Libraries
import concurrent.futures
import json
import numpy as np
import pandas as pd
import requests
import tensorflow as tf
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import joblib
from concurrent.futures import ThreadPoolExecutor

In [ ]:
# Define GraphQL Query and Headers

# GraphQL query for fetching user contest data
query = """
query userContestRankingInfo($username: String!) {
    userContestRankingHistory(username: $username) {
        attended
        rating
        ranking
    }
}
"""

# Headers for the GraphQL request
headers = {"Content-Type": "application/json"}

In [ ]:
session = requests.Session()

def read_usernames_from_json(file_path, number_of_usernames=40000):
    with open(file_path, 'r') as file:
        all_usernames = json.load(file)
        return all_usernames[:number_of_usernames]

def fetch_data(username):
    response = requests.post(
        "https://leetcode.com/graphql",
        headers=headers,
        json={"query": query, "variables": {"username": username}}
    )
    if response.status_code == 200:
        return response.json().get("data", {}).get("userContestRankingHistory", [])
    else:
        print(f"Error fetching data for username {username}: {response.status_code}")
        return []  # Return an empty list in case of error


In [ ]:
def process_data(contests):
    data = []
    rating=1500
    for contest in contests:
        if contest["attended"]:
            prev_rating = rating
            rating = contest["rating"]
            ranking = contest["ranking"]
            data.append([prev_rating, ranking, prev_rating / ranking if ranking != 0 else 0, rating - prev_rating])
    return data

def process_batch_parallel(usernames_batch, max_workers=100):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Use map to execute the function in parallel
        results = executor.map(fetch_data, usernames_batch)

    data = []
    for contests in results:
        if contests:  # Check if contests is not empty
            data.extend(process_data(contests))

    return data


usernames = read_usernames_from_json('usernames.json')

all_data = []
batch_size = 1000
for i in range(0, len(usernames), batch_size):
    batch_data = process_batch_parallel(usernames[i:i + batch_size])
    all_data.extend(batch_data)
    time.sleep(1)  # Sleep to respect API rate limits


In [ ]:
df = pd.DataFrame(all_data, columns=['input1', 'input2', 'input3', 'output'])
print(df)
df.to_json('data.json', orient='records', lines=True)

X = df.iloc[:, :-1].values
y = df['output'].values

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, 'scaler.save')
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3)

          input1  input2     input3   output
0       1500.000     110  13.636364  320.626
1       1820.626      45  40.458356  199.425
2       2020.051      69  29.276101  102.787
3       2122.838     119  17.838975   73.217
4       2196.055     384   5.718893   19.398
...          ...     ...        ...      ...
170092  1500.000     563   2.664298  271.417
170093  1771.417     158  11.211500  204.940
170094  1976.357     604   3.272114   73.783
170095  2050.140     306   6.699804   85.929
170096  2136.069     191  11.183607   68.161

[170097 rows x 4 columns]


In [ ]:
# Ensure using a GPU if available
with tf.device('/device:GPU:0'):

    # Define LSTM model with cuDNN compatible configurations
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(500, activation='tanh', recurrent_activation='sigmoid', input_shape=(1, 3)),
        tf.keras.layers.Dense(1)
    ])

    # Set a custom learning rate
    learning_rate = 0.01
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32)

    # Evaluate the model
    loss = model.evaluate(X_test, y_test)
    print("Test Loss:", loss)

    # Save the model in the recommended format (.keras)
    model.save('model.keras')


Epoch 1/10
3721/3721 [==============================] - 18s 4ms/step - loss: 794.2309
Epoch 2/10
3721/3721 [==============================] - 16s 4ms/step - loss: 725.9354
Epoch 3/10
3721/3721 [==============================] - 15s 4ms/step - loss: 717.2713
Epoch 4/10
3721/3721 [==============================] - 14s 4ms/step - loss: 709.9019
Epoch 5/10
3721/3721 [==============================] - 15s 4ms/step - loss: 698.6470
Epoch 6/10
3721/3721 [==============================] - 14s 4ms/step - loss: 683.4938
Epoch 7/10
3721/3721 [==============================] - 14s 4ms/step - loss: 678.0026
Epoch 8/10
3721/3721 [==============================] - 14s 4ms/step - loss: 673.4736
Epoch 9/10
3721/3721 [==============================] - 15s 4ms/step - loss: 671.3234
Epoch 10/10
1595/1595 [==============================] - 4s 2ms/step - loss: 637.5876
Test Loss: 637.587646484375


In [ ]:
!pip install tensorflowjs

import tensorflow as tf
import tensorflowjs as tfjs

# Load your previously saved Keras model
model_path = '/content/model.keras'  # Replace with the path to your saved model
model = tf.keras.models.load_model(model_path)

# Directory where the TensorFlow.js model will be saved
output_dir = '/content/tfjs_model'

# Convert the model
tfjs.converters.save_keras_model(model, output_dir)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import numpy as np
import tensorflow as tf
import joblib

# Load the trained model and the scaler
model = tf.keras.models.load_model('/content/model.keras')
scaler = joblib.load('/content/scaler.save')

def get_user_input():
    """
    Prompts the user to enter the required input values.
    """
    print("Enter the input values:")
    input1 = float(input("Enter your previous rating: "))
    input2 = float(input("Enter your ranking: "))
    input3 = input1 / input2 if input2 != 0 else 0

    return np.array([[input1, input2, input3]])

def make_prediction(input_data):
    """
    Normalizes the input data and makes a prediction.
    """
    input_scaled = scaler.transform(input_data)
    input_scaled = input_scaled.reshape((input_scaled.shape[0], 1, input_scaled.shape[1]))
    prediction = model.predict(input_scaled)
    return prediction[0][0]

def main():
    user_input = get_user_input()
    prediction = make_prediction(user_input)
    print(f"Predicted change in rating: {prediction}")

if __name__ == "__main__":
    main()
